# Data description

#!sed 's/","/\|/g' DiccionarioDatos.csv | sed 's/"/|/g'

|Variable| Descripción|
|---|---|
|Cod Suer|Codigo Lote|
|ESTADO|Plantilla=1er.corte, SOCA>=2 cortes|
|ZAE|Zona agroecologica |
|ZC|Zona Climatica|
|Suer|Codigo Lote|
|Edad|edad de cosecha(meses)|
|Variedad|Codigo de la Variedad|
|Sac|Sacarosa %|
|Rend|Aucar x tonelada de caña %|
|Fecha|Fecha inicio corte|
|CICLO|No. de renovaciones|
|Fecha Cierre|Fecha terminacion corte|
|TCH REAL|Tons de caña por hectarea|
|Cort Act|No. corte actual|
|TCHM R|Tons de caña por hectarea por mes|
|Area Actual|Area en hectareas|
|Caña Ent|Total de toneladas de caña (TTC)|
|Año|Año de corte|
|Mes|Mes de corte|
|F C Ant|Fecha de corte anterior|
|30 DAC|Lluvia 30 dias antes de corte anterior|
|PST 0-4 Meses|Lluvia acumulada despues de corte o siembra hasta el 4to mes |
|PRC 4-9 Meses|Lluvia acumuladad entre 4 y 9 meses|
|0-15 DDC|lluvia entre 0 y 15 dias despues de corte|
|16-30 DDC|Lluvia entre 16 y 30 dias despues de corte|
|31-45 DDC||
|46-60 DDC||
|61-75 DDC||
|76-90 DDC||
|91-105 DDC||
|106-120 DDC||
|121-135 DDC||
|136-150 DDC||
|151-165 DDC||
|166-180 DDC||
|181-195 DDC||
|196-210 DDC||
|211-225 DDC||
|226-240 DDC||
|241-255 DDC||
|256-270 DDC||
|271-285 DDC||
|30 DAC Act|Lluvia 30 dias antes de corte actual|

## import the libraries

In [53]:
# import the necessary packages
import pandas as pd
import numpy as np
import argparse
import cv2
import glob
from pandas import DataFrame
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import datetime


In [133]:
data=pd.read_csv("luvias05_18cb1_3.csv")

In [134]:
data.columns

Index(['codSuer', 'ESTADO', 'ZAE', 'ZC', 'Suer', 'Edad', 'Variedad', 'Sac',
       'Rend', 'Fecha', 'CICLO', 'Fecha_Cierre', 'TCH_REAL', 'Cort_Act',
       'TCHM_R', 'Area_Actual', 'Cana_Ent', 'Anio', 'Mes', 'F_C_Ant', '30_DAC',
       'PS_0-4_Meses', 'PRC_4-9_Meses', '0-15_DDC', '16-30_DDC', '31-45_DDC',
       '46-60_DDC', '61-75_DDC', '76-90_DDC', '91-105_DDC', '106-120_DDC',
       '121-135_DDC', '136-150_DDC', '151-165_DDC', '166-180_DDC',
       '181-195_DDC', '196-210_DDC', '211-225_DDC', '226-240_DDC',
       '241-255_DDC', '256-270_DDC', '271-285_DDC', '30_DAC_Act'],
      dtype='object')

## Convert to data types

In [139]:
from pandas.api.types import CategoricalDtype

numericColumns = ["Edad", "Sac", "Rend", 'TCH_REAL', 'Cort_Act','TCH_REAL','TCHM_R', 
                  'Area_Actual', 'Cana_Ent', 'Anio', 'Mes', '30_DAC', 'PS_0-4_Meses', 'PRC_4-9_Meses', 
                  '0-15_DDC', '16-30_DDC', '31-45_DDC','46-60_DDC', '61-75_DDC',
                  '76-90_DDC', '91-105_DDC', '106-120_DDC', '121-135_DDC', 
                  '136-150_DDC', '151-165_DDC', '166-180_DDC', '181-195_DDC',
                  '196-210_DDC', '211-225_DDC', '226-240_DDC', '241-255_DDC', 
                  '256-270_DDC', '271-285_DDC', '30_DAC_Act']

factorColumns = ["codSuer", "ESTADO", "ZAE", "ZC", "Suer", "Variedad" ]
dateColumns = ["Fecha", "Fecha_Cierre", "F_C_Ant" ]
levelColumns = ["CICLO",  "Cort_Act"]
target = ["TCH_REAL", "TCH_R", "Cana_Ent", "Rend", "Sac"]

# Assign to the index the corresponding column
#data.index = data["EmployeeNumber"].values
# Convert to numeric attributes
# Nothing to do in python. Numeric is the dafault
for col in numericColumns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
    

# loop to change each column to category type
for col in factorColumns:
    cat_type = CategoricalDtype(categories = None, ordered = False)
    data[col] = data[col].astype(dtype = cat_type)

# Conver to levels / Ordinals
for col in levelColumns:
    cat_type = CategoricalDtype(categories = None, ordered = True)
    data[col] = data[col].astype(dtype = cat_type)

# Conver to st ring/ Date
for col in dateColumns:
    data[col] =  pd.to_datetime(data[col], format='%d/%m/%Y')

## Seleccionamos las variables de interés

In [140]:
numericColumns = ["Edad", 'Cort_Act', 'Area_Actual', 'Mes', '30_DAC', 'PS_0-4_Meses', 'PRC_4-9_Meses', 
                  '0-15_DDC', '16-30_DDC', '31-45_DDC','46-60_DDC', '61-75_DDC',
                  '76-90_DDC', '91-105_DDC', '106-120_DDC', '121-135_DDC', 
                  '136-150_DDC', '151-165_DDC', '166-180_DDC', '181-195_DDC',
                  '196-210_DDC', '211-225_DDC', '226-240_DDC', '241-255_DDC', 
                  '256-270_DDC', '271-285_DDC', '30_DAC_Act']

factorColumns = ["ESTADO", "ZAE", "ZC", "Variedad"]
dateColumns = ["Fecha", "Fecha_Cierre", "F_C_Ant" ]
levelColumns = ["CICLO",  "Cort_Act"]
target = ["TCH_REAL", "TCH_R", "Cana_Ent"]

dataX = data.loc[:,numericColumns + factorColumns + levelColumns]
dataY = data[target[0]].values

#Esta variable nos dice cuantos días de descanso hubo entre cosechas
dataX["descanso"] = np.floor((data["Fecha"] - data["F_C_Ant"]).dt.days - data["Edad"] * 30)

In [147]:
dataX.head()


## Preparar datos para clasificación. Pasar factores a dummy y completar faltantes con la media de cada característica

In [142]:
data2 = dataX.copy()

for catAtt in factorColumns:
    dummies = pd.get_dummies(data2[catAtt], prefix = catAtt)
    data2 = pd.concat([data2.drop(catAtt, axis = 1), dummies], axis = 1)
    

# Fill NAN in numerical attributes
foo = data2.isnull().sum()
index = 0
for f in foo:
    if f > 0:
        colname = data2.columns[index]
        print(colname)
        data2[colname] = data2[colname].fillna(data2[colname].mean())
    index = index + 1


In [155]:
data2.columns

Index(['Edad', 'Cort_Act', 'Area_Actual', 'Mes', '30_DAC', 'PS_0-4_Meses',
       'PRC_4-9_Meses', '0-15_DDC', '16-30_DDC', '31-45_DDC',
       ...
       'Variedad_MISCELANEOS', 'Variedad_MZC74275', 'Variedad_MZC8211',
       'Variedad_PR61632', 'Variedad_RB732223', 'Variedad_RD7511',
       'Variedad_SP61632', 'Variedad_SP716949', 'Variedad_V7151',
       'Variedad_VARIEDADESEXP.'],
      dtype='object', length=193)

In [180]:
# Modelo de predicción

# Se divide el set de datos en dos conjuntos train y test
y = dataY
x = data2
trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.3)

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# NN is sensitive to data scale. We must normilize
scaler = StandardScaler()  
trainXX = trainX.copy()
testXX = testX.copy()
# Don't cheat - fit only on training data
scaler.fit(trainX)  
trainXX = scaler.transform(trainXX)  
# apply same transformation to test data
testXX = scaler.transform(testXX)  

model3 = MLPRegressor(solver='adam', alpha=1e-3, activation = 'logistic', max_iter=30000,
                     hidden_layer_sizes = (30, 10, 3), verbose = True)

model3.fit(trainXX, trainY)                         

#Print the score on the train data
print("On training")
print(model3.score(trainXX, trainY))
#print(confusion_matrix(model3.predict(trainXX), trainY))

#Print the score on the test data
print("\nOn test")
print(model3.score(testXX, testY))
#print(confusion_matrix(model3.predict(testXX), testY))

Iteration 1, loss = 5274.37803228
Iteration 2, loss = 5244.90843063
Iteration 3, loss = 5223.22196142
Iteration 4, loss = 5205.36698693
Iteration 5, loss = 5189.24878794
Iteration 6, loss = 5173.96496418
Iteration 7, loss = 5159.09224175
Iteration 8, loss = 5144.34808969
Iteration 9, loss = 5129.14527838
Iteration 10, loss = 5109.48521919
Iteration 11, loss = 5075.40165564
Iteration 12, loss = 5040.15227607
Iteration 13, loss = 5010.47488505
Iteration 14, loss = 4983.75030095
Iteration 15, loss = 4958.69152529
Iteration 16, loss = 4934.72908282
Iteration 17, loss = 4911.54054969
Iteration 18, loss = 4888.94168819
Iteration 19, loss = 4866.80580165
Iteration 20, loss = 4845.05237461
Iteration 21, loss = 4823.61284776
Iteration 22, loss = 4802.44269941
Iteration 23, loss = 4781.51125480
Iteration 24, loss = 4760.78751375
Iteration 25, loss = 4740.25467073
Iteration 26, loss = 4719.88098822
Iteration 27, loss = 4699.66997544
Iteration 28, loss = 4679.59716026
Iteration 29, loss = 4659.657

Iteration 232, loss = 1108.85823086
Iteration 233, loss = 1099.36768505
Iteration 234, loss = 1089.95461470
Iteration 235, loss = 1080.60144365
Iteration 236, loss = 1071.33150619
Iteration 237, loss = 1062.12594648
Iteration 238, loss = 1052.99696861
Iteration 239, loss = 1043.93338327
Iteration 240, loss = 1034.95277822
Iteration 241, loss = 1026.03610012
Iteration 242, loss = 1017.19963471
Iteration 243, loss = 1008.42125120
Iteration 244, loss = 999.73083028
Iteration 245, loss = 991.10242844
Iteration 246, loss = 982.54740693
Iteration 247, loss = 974.06201904
Iteration 248, loss = 965.64745893
Iteration 249, loss = 957.30650147
Iteration 250, loss = 949.02880497
Iteration 251, loss = 940.83789816
Iteration 252, loss = 932.70396997
Iteration 253, loss = 924.64690914
Iteration 254, loss = 916.66924983
Iteration 255, loss = 908.75073148
Iteration 256, loss = 900.90734094
Iteration 257, loss = 893.13249005
Iteration 258, loss = 885.42878735
Iteration 259, loss = 877.80158244
Iteratio

Iteration 466, loss = 315.09512778
Iteration 467, loss = 313.69285175
Iteration 468, loss = 312.30434970
Iteration 469, loss = 311.11627004
Iteration 470, loss = 309.89583044
Iteration 471, loss = 308.75112033
Iteration 472, loss = 307.62639091
Iteration 473, loss = 306.48463948
Iteration 474, loss = 305.49331084
Iteration 475, loss = 304.37202532
Iteration 476, loss = 303.42903907
Iteration 477, loss = 302.49374136
Iteration 478, loss = 301.49773409
Iteration 479, loss = 300.73278704
Iteration 480, loss = 299.72166842
Iteration 481, loss = 298.82288147
Iteration 482, loss = 298.09873165
Iteration 483, loss = 297.21300211
Iteration 484, loss = 296.44434799
Iteration 485, loss = 295.65704598
Iteration 486, loss = 294.81789689
Iteration 487, loss = 293.97818064
Iteration 488, loss = 293.22596301
Iteration 489, loss = 292.40300782
Iteration 490, loss = 291.57639316
Iteration 491, loss = 290.83632756
Iteration 492, loss = 290.06589493
Iteration 493, loss = 289.31994294
Iteration 494, loss 

Iteration 702, loss = 210.54443675
Iteration 703, loss = 210.36569758
Iteration 704, loss = 210.10023875
Iteration 705, loss = 209.90876289
Iteration 706, loss = 209.76630507
Iteration 707, loss = 209.61873205
Iteration 708, loss = 209.59187934
Iteration 709, loss = 209.32814171
Iteration 710, loss = 209.09189946
Iteration 711, loss = 208.96311624
Iteration 712, loss = 208.89662670
Iteration 713, loss = 208.54622036
Iteration 714, loss = 208.43599711
Iteration 715, loss = 208.19557151
Iteration 716, loss = 208.00043092
Iteration 717, loss = 208.07301903
Iteration 718, loss = 207.67212118
Iteration 719, loss = 207.61956063
Iteration 720, loss = 207.47147520
Iteration 721, loss = 207.25450720
Iteration 722, loss = 206.97800958
Iteration 723, loss = 206.73595016
Iteration 724, loss = 206.55559118
Iteration 725, loss = 206.59641106
Iteration 726, loss = 206.51222326
Iteration 727, loss = 206.29128674
Iteration 728, loss = 205.84554605
Iteration 729, loss = 205.70938546
Iteration 730, loss 

Iteration 938, loss = 185.84845803
Iteration 939, loss = 185.89653563
Iteration 940, loss = 185.83276787
Iteration 941, loss = 185.89484243
Iteration 942, loss = 185.82511455
Iteration 943, loss = 185.77263436
Iteration 944, loss = 185.62651493
Iteration 945, loss = 185.72752994
Iteration 946, loss = 185.48290223
Iteration 947, loss = 185.58100715
Iteration 948, loss = 185.45047639
Iteration 949, loss = 185.44727176
Iteration 950, loss = 185.27840992
Iteration 951, loss = 185.43177931
Iteration 952, loss = 185.25829444
Iteration 953, loss = 185.20218968
Iteration 954, loss = 185.17162495
Iteration 955, loss = 185.06836868
Iteration 956, loss = 184.92598342
Iteration 957, loss = 184.94317937
Iteration 958, loss = 184.67285454
Iteration 959, loss = 184.94143283
Iteration 960, loss = 184.92616796
Iteration 961, loss = 184.80797836
Iteration 962, loss = 184.79446818
Iteration 963, loss = 184.72295136
Iteration 964, loss = 184.55460783
Iteration 965, loss = 184.44166028
Iteration 966, loss 

Iteration 1171, loss = 175.36446243
Iteration 1172, loss = 175.33420676
Iteration 1173, loss = 175.33750469
Iteration 1174, loss = 175.21787936
Iteration 1175, loss = 175.11076478
Iteration 1176, loss = 175.39625463
Iteration 1177, loss = 175.18181714
Iteration 1178, loss = 175.17195046
Iteration 1179, loss = 174.94121112
Iteration 1180, loss = 174.97447675
Iteration 1181, loss = 175.25810976
Iteration 1182, loss = 175.00079894
Iteration 1183, loss = 174.88147594
Iteration 1184, loss = 174.73943959
Iteration 1185, loss = 174.79821300
Iteration 1186, loss = 174.85230656
Iteration 1187, loss = 174.71353322
Iteration 1188, loss = 174.63484819
Iteration 1189, loss = 174.80843295
Iteration 1190, loss = 174.84306048
Iteration 1191, loss = 174.64284098
Iteration 1192, loss = 174.67676269
Iteration 1193, loss = 174.65423953
Iteration 1194, loss = 174.64222539
Iteration 1195, loss = 174.57181205
Iteration 1196, loss = 174.57531723
Iteration 1197, loss = 174.46837549
Iteration 1198, loss = 174.5

Iteration 1399, loss = 168.66513350
Iteration 1400, loss = 168.80846359
Iteration 1401, loss = 168.73835098
Iteration 1402, loss = 168.83995860
Iteration 1403, loss = 168.58832104
Iteration 1404, loss = 168.49985130
Iteration 1405, loss = 168.55835555
Iteration 1406, loss = 168.71967042
Iteration 1407, loss = 168.85369667
Iteration 1408, loss = 168.46992302
Iteration 1409, loss = 168.43847376
Iteration 1410, loss = 168.52101794
Iteration 1411, loss = 168.42859236
Iteration 1412, loss = 168.54358359
Iteration 1413, loss = 168.36400836
Iteration 1414, loss = 168.53286207
Iteration 1415, loss = 168.33626966
Iteration 1416, loss = 168.32457226
Iteration 1417, loss = 168.37963359
Iteration 1418, loss = 168.14651336
Iteration 1419, loss = 168.41778183
Iteration 1420, loss = 168.20729665
Iteration 1421, loss = 168.24068270
Iteration 1422, loss = 168.33164310
Iteration 1423, loss = 168.16784093
Iteration 1424, loss = 168.06361763
Iteration 1425, loss = 168.14912224
Iteration 1426, loss = 168.1

Iteration 1627, loss = 164.36535658
Iteration 1628, loss = 164.39366436
Iteration 1629, loss = 164.41966173
Iteration 1630, loss = 164.39769935
Iteration 1631, loss = 164.34938229
Iteration 1632, loss = 164.42958776
Iteration 1633, loss = 164.48033952
Iteration 1634, loss = 164.49082429
Iteration 1635, loss = 164.29999708
Iteration 1636, loss = 164.31397481
Iteration 1637, loss = 164.38681610
Iteration 1638, loss = 164.31775871
Iteration 1639, loss = 164.24225768
Iteration 1640, loss = 164.21474574
Iteration 1641, loss = 164.26632143
Iteration 1642, loss = 164.13268034
Iteration 1643, loss = 164.37323834
Iteration 1644, loss = 164.29660640
Iteration 1645, loss = 164.12164188
Iteration 1646, loss = 164.18015355
Iteration 1647, loss = 164.09927634
Iteration 1648, loss = 164.21507395
Iteration 1649, loss = 164.14898247
Iteration 1650, loss = 164.05702214
Iteration 1651, loss = 163.99634088
Iteration 1652, loss = 163.97157991
Iteration 1653, loss = 164.21812520
Iteration 1654, loss = 164.1

Iteration 1855, loss = 161.00063426
Iteration 1856, loss = 161.18027951
Iteration 1857, loss = 161.00858307
Iteration 1858, loss = 161.06330463
Iteration 1859, loss = 160.99244938
Iteration 1860, loss = 160.88683797
Iteration 1861, loss = 160.98979095
Iteration 1862, loss = 161.08828254
Iteration 1863, loss = 161.00598640
Iteration 1864, loss = 160.94795457
Iteration 1865, loss = 161.04528834
Iteration 1866, loss = 161.08382255
Iteration 1867, loss = 161.04344935
Iteration 1868, loss = 160.81976946
Iteration 1869, loss = 161.00580658
Iteration 1870, loss = 161.08013431
Iteration 1871, loss = 160.90076503
Iteration 1872, loss = 160.87284194
Iteration 1873, loss = 160.81494749
Iteration 1874, loss = 160.85127414
Iteration 1875, loss = 160.86688255
Iteration 1876, loss = 160.76591842
Iteration 1877, loss = 160.89036788
Iteration 1878, loss = 160.87385400
Iteration 1879, loss = 160.91539268
Iteration 1880, loss = 160.82213064
Iteration 1881, loss = 160.80456459
Iteration 1882, loss = 160.8

In [181]:
model3.predict(trainXX)

array([112.2236029 ,  99.78645435,  89.084283  , ...,  83.19626884,
        68.57873784, 118.23767323])

In [182]:
trainY

array([124.49, 108.63,  92.35, ...,  94.52,  53.55, 154.12])

In [183]:
model3.predict(testXX)

array([147.76694845,  80.97455346, 107.75777497, ...,  92.63873289,
       110.64435043, 131.03390468])

In [184]:
testY

array([117.04,  31.26, 102.88, ...,  93.08, 111.77, 116.33])

In [185]:
plt.(model3.predict(testXX), testY)

NameError: name 'plot' is not defined

In [186]:
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'